# Temporal Fusion Transformer

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


%cd /content/gdrive/My Drive/Pump Data/
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Pump Data
allpumps_nbeats.pth.tar       __pycache__
allpumps_tcn_v1.pth.tar       tcn_valid_results.csv
allpumps_transformer.pth.tar  test_results.csv
data			      transformers_valid_results.csv
nbeats_valid_results.csv      utils.py


In [4]:
!pip install matplotlib==3.1.3
!pip install pytorch-forecasting
!pip install darts
!pip install pyyaml==5.4.1

  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.0
    Uninstalling matplotlib-3.5.0:
      Successfully uninstalled matplotlib-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-forecasting 0.9.1 requires scikit-learn<0.25.0,>=0.24.0, but you have scikit-learn 1.0.1 which is incompatible.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.
darts 0.13.1 requires matplotlib>=3.4.0, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
darts 0.13.1 requires matplotlib>=3.4.0, but you have matplotlib 3.1.3 which is incompatible.
darts 0.13.1 requires scikit-learn>=1.0.1, but you have scikit-learn 0.24.2 which is incompatible.


  Using cached scikit_learn-1.0.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.2 MB)
  Using cached matplotlib-3.5.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-forecasting 0.9.1 requires scikit-learn<0.25.0,>=0.24.0, but you have scikit-learn 1.0.1 which is incompatible.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you hav

     |████████████████████████████████| 636 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-forecasting 0.9.1 requires scikit-learn<0.25.0,>=0.24.0, but you have scikit-learn 1.0.1 which is incompatible.


в этом ноутбуке смотрим реализацию https://pytorch-forecasting.readthedocs.io/en/latest/api/pytorch_forecasting.models.temporal_fusion_transformer.TemporalFusionTransformer.html и считаем насколько они хорошо работают

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer



In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, Theta, TCNModel
from darts.metrics import mape, rmse, r2_score
from darts.utils.statistics import check_seasonality, plot_acf, plot_hist
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from darts.datasets import SunspotsDataset, AirPassengersDataset
from darts import TimeSeries
from darts.models import TCNModel, RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, r2_score
from darts.utils.missing_values import fill_missing_values
from darts.datasets import AirPassengersDataset, SunspotsDataset, EnergyDataset
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller, Mapper, InvertibleMapper


OSError: dlopen(/Users/a1/Desktop/oil-project/.venv/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/a1/Desktop/oil-project/.venv/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [11]:
FC_START = '2019-07-25 00:00:00'   # period at which to split training and validation dataset


In [4]:
data_address = 'data'
training = pd.read_csv(data_address +'/cfp_dataset_v1.csv', index_col=False)
training.drop(['Unnamed: 0', 'MIDUPT1138P2300058', 'SHORTUPT1138P2300058', 'LONGUPT1138P2300058', 'DMIDT1138P2300058',  'DLONGT1138P2300058', 'UNIXDT', 'UUID'], axis=1, inplace=True)
training.columns

Index(['WELL_ID', 'DT', 'T1138P6000096', 'T1138P6000315', 'DMIDT1138P4000064',
       'DSHORTT1138P4000064', 'DLONGT1138P4000064', 'DMIDT1138P2600012',
       'DSHORTT1138P2600012', 'DLONGT1138P2600012', 'DMIDT1205P2300000',
       'DSHORTT1205P2300000', 'DLONGT1205P2300000', 'T1205P2300000',
       'T1138P4000064', 'T1138P2600012', 'T1138P600050', 'T1013P500399',
       'DSHORTT1138P2300058'],
      dtype='object')

In [32]:

pump_encoder = LabelBinarizer()
transformed = pump_encoder.fit_transform(training.WELL_ID)
ohe_df = pd.DataFrame(transformed)
ohe_df.columns = pump_encoder.classes_
ohe_df.columns = ohe_df.columns.astype(str)

# training_oh = pd.concat([training, ohe_df], axis=1).drop(['WELL_ID'], axis=1)
training_oh = pd.concat([training, ohe_df], axis=1)
# training_oh
training_oh.columns = training_oh.columns.astype(str)
training_oh.columns

Index(['WELL_ID', 'DT', 'T1138P6000096', 'T1138P6000315', 'DMIDT1138P4000064',
       'DSHORTT1138P4000064', 'DLONGT1138P4000064', 'DMIDT1138P2600012',
       'DSHORTT1138P2600012', 'DLONGT1138P2600012', 'DMIDT1205P2300000',
       'DSHORTT1205P2300000', 'DLONGT1205P2300000', 'T1205P2300000',
       'T1138P4000064', 'T1138P2600012', 'T1138P600050', 'T1013P500399',
       'DSHORTT1138P2300058', '226000188', '226000198', '226000297',
       '226001846', '226001947', '226002142', '226002148', '226002307',
       '226002416', '226002603', '226002627', '226002861', '226002998',
       '226003074', '226003080', '226003316', '226003330'],
      dtype='object')

In [34]:
def get_pump_names(df):
  """
  вытаскиваем set из колонок по df, который подали
  """
  values = df.columns
  return set(values)


def split_df_by_pumps(input_df, ohe_df):
  """
  input_df: дф, в котором есть колонки onehot с насовами
  ohe_df: дф onehot для насосов
  выдает 17 штук train set
  """

  pump_ids = get_pump_names(ohe_df)
  df_list = list()

  for pump in pump_ids:
    df_list.append(input_df[input_df[pump] == 1])

  return df_list


def get_one_pump_train_date(inp_df):
  onehot_series_0 = TimeSeries.from_dataframe(inp_df, 'DT', fill_missing_dates=True, freq='5T')

  scaler = Scaler()
  ts = scaler.fit_transform(onehot_series_0)  # scale the whole time series not caring about train/val split...
  filler = MissingValuesFiller()
  ts = filler.transform(ts, method='linear')
  target = ts['DSHORTT1138P2300058']
  # Create training and validation sets:
  train_target, val_target = target.split_after(pd.Timestamp(FC_START))

  covariates = ts[['T1138P6000096', 'T1138P6000315', 'DMIDT1138P4000064',
        'DSHORTT1138P4000064', 'DLONGT1138P4000064', 'DMIDT1138P2600012',
        'DSHORTT1138P2600012', 'DLONGT1138P2600012', 'DMIDT1205P2300000',
        'DSHORTT1205P2300000', 'DLONGT1205P2300000', 'T1205P2300000',
        'T1138P4000064', 'T1138P2600012', 'T1138P600050', 'T1013P500399', 
       'DSHORTT1138P2300058',             '226000188', '226000198', '226000297',
       '226001846', '226001947', '226002142', '226002148', '226002307',
       '226002416', '226002603', '226002627', '226002861', '226002998',
       '226003074', '226003080', '226003316', '226003330']]

  train_cov, val_cov = covariates.split_after(pd.Timestamp(FC_START))
  print(len(train_cov), len(train_target), len(val_cov), len(val_target))
  return train_cov, train_target, val_cov, val_target

def get_all_pump_training_date(df_all_pumps):
  train_cov_all, train_target_all, val_cov_all, val_target_all = [], [], [], []
  for pump_df in df_all_pumps:
    iter_train, iter_target, iter_val, iter_val_target = get_one_pump_train_date(pump_df)
    train_cov_all.append(iter_train)
    train_target_all.append(iter_target)
    val_cov_all.append(iter_val)
    val_target_all.append(iter_val_target)
  return train_cov_all, train_target_all, val_cov_all, val_target_all


In [35]:
all_pumps_df[0].columns

Index([            'WELL_ID',                  'DT',       'T1138P6000096',
             'T1138P6000315',   'DMIDT1138P4000064', 'DSHORTT1138P4000064',
        'DLONGT1138P4000064',   'DMIDT1138P2600012', 'DSHORTT1138P2600012',
        'DLONGT1138P2600012',   'DMIDT1205P2300000', 'DSHORTT1205P2300000',
        'DLONGT1205P2300000',       'T1205P2300000',       'T1138P4000064',
             'T1138P2600012',        'T1138P600050',        'T1013P500399',
       'DSHORTT1138P2300058',             226000188,             226000198,
                   226000297,             226001846,             226001947,
                   226002142,             226002148,             226002307,
                   226002416,             226002603,             226002627,
                   226002861,             226002998,             226003074,
                   226003080,             226003316,             226003330],
      dtype='object')

In [37]:
training_oh

,WELL_ID,DT,T1138P6000096,T1138P6000315,DMIDT1138P4000064,DSHORTT1138P4000064,DLONGT1138P4000064,DMIDT1138P2600012,DSHORTT1138P2600012,DLONGT1138P2600012,DMIDT1205P2300000,DSHORTT1205P2300000,DLONGT1205P2300000,T1205P2300000,T1138P4000064,T1138P2600012,T1138P600050,T1013P500399,DSHORTT1138P2300058,226000188,226000198,226000297,226001846,226001947,226002142,226002148,226002307,226002416,226002603,226002627,226002861,226002998,226003074,226003080,226003316,226003330
0,226003330,2019-07-31 23:55:00,300,0,-2.150755,0.0,-2.290611,-0.279070,0.0,-0.313752,-0.001102,0.000587,-0.000245,1.168133,70,19.6,0.759,NaN,-0.1344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,226003330,2019-07-31 23:50:00,0,0,-2.172480,0.0,-2.292904,-0.281889,0.0,-0.314066,-0.001172,0.000000,-0.000251,1.167644,70,19.6,0.761,NaN,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,226003330,2019-07-31 23:45:00,0,28500,-2.172480,0.0,-2.292904,-0.281889,0.0,-0.314066,-0.001172,0.000000,-0.000251,1.161290,0,0.0,0.000,NaN,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,226003330,2019-07-31 23:40:00,0,28200,-2.172480,0.0,-2.292904,-0.281889,0.0,-0.314066,-0.001172,0.000000,-0.000251,1.160802,0,0.0,0.000,NaN,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,226003330,2019-07-31 23:35:00,0,27900,-2.172480,0.0,-2.292904,-0.281889,0.0,-0.314066,-0.001172,0.000000,-0.000251,1.162268,0,0.0,0.000,NaN,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160257,226000188,2019-06-27 07:00:00,0,14400,-3.232999,0.0,-4.138484,-0.423734,0.0,-0.656976,0.001927,0.000000,0.000129,1.642718,0,0.0,0.000,NaN,0.0000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
160258,226000188,2019-06-27 06:55:00,0,14100,-3.232999,0.0,-4.138484,-0.423734,0.0,-0.656976,0.001927,0.000000,0.000129,1.648583,0,0.0,0.000,NaN,0.0000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
160259,226000188,2019-06-27 06:50:00,0,13800,-3.232999,0.0,-4.138484,-0.423734,0.0,-0.656976,0.001927,0.000000,0.000129,1.650049,0,0.0,0.000,NaN,0.0000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
160260,226000188,2019-06-27 06:45:00,0,13500,-3.232999,0.0,-4.138484,-0.423734,0.0,-0.656976,0.001927,0.000000,0.000129,1.681818,0,0.0,0.000,NaN,0.0000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
all_pumps_df = split_df_by_pumps(training_oh, ohe_df)
train_cov_v3, train_target_v3, val_cov_v3, val_target_v3= get_all_pump_training_date(all_pumps_df)

7985 7985 2015 2015
7985 7985 2015 2015
1165 1165 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015
7985 7985 2015 2015


### todo#

1.   сделать прогон основного кода
2.   если не валится, запустить этот ноутбук
3.   если все ок, то сюда добавить
4. продолжить наполнять даннные в этот даталоадер
4. сохранить данные на диск, считать в пандас в нем объединить строки




In [20]:
train_cov_v3[0].to_csv('pump1.gz')


In [21]:
pd.read_csv('pump1.gz')

,DT,T1138P6000096,T1138P6000315,DMIDT1138P4000064,DSHORTT1138P4000064,DLONGT1138P4000064,DMIDT1138P2600012,DSHORTT1138P2600012,DLONGT1138P2600012,DMIDT1205P2300000,DSHORTT1205P2300000,DLONGT1205P2300000,T1205P2300000,T1138P4000064,T1138P2600012,T1138P600050,T1013P500399
0,2019-06-27 06:40:00,0.0,0.195122,0.180549,0.849074,0.032678,0.409191,0.888526,0.027582,0.025863,0.091077,0.000743,0.026255,0.0,0.0,0.0,0.0
1,2019-06-27 06:45:00,0.0,0.198606,0.180549,0.849074,0.032678,0.409191,0.888526,0.027582,0.025863,0.091077,0.000743,0.021602,0.0,0.0,0.0,0.0
2,2019-06-27 06:50:00,0.0,0.202091,0.180549,0.849074,0.032678,0.409191,0.888526,0.027582,0.025863,0.091077,0.000743,0.024593,0.0,0.0,0.0,0.0
3,2019-06-27 06:55:00,0.0,0.205575,0.180549,0.849074,0.032678,0.409191,0.888526,0.027582,0.025863,0.091077,0.000743,0.023430,0.0,0.0,0.0,0.0
4,2019-06-27 07:00:00,0.0,0.209059,0.180549,0.849074,0.032678,0.409191,0.888526,0.027582,0.025863,0.091077,0.000743,0.024094,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7980,2019-07-24 23:40:00,0.0,0.205575,0.404208,0.849074,0.866243,0.422577,0.888526,0.523404,0.038568,0.091077,0.139263,0.023762,0.0,0.0,0.0,0.0
7981,2019-07-24 23:45:00,0.0,0.209059,0.404208,0.849074,0.866243,0.422577,0.888526,0.523404,0.038568,0.091077,0.139263,0.021768,0.0,0.0,0.0,0.0
7982,2019-07-24 23:50:00,0.0,0.212544,0.404208,0.849074,0.866243,0.422577,0.888526,0.523404,0.038568,0.091077,0.139263,0.021602,0.0,0.0,0.0,0.0
7983,2019-07-24 23:55:00,0.0,0.216028,0.404208,0.849074,0.866243,0.422577,0.888526,0.523404,0.038568,0.091077,0.139263,0.019109,0.0,0.0,0.0,0.0


In [ ]:

# define dataset
max_encoder_length = 144
max_prediction_length = 1
training_cutoff = "2019-07-25" 


this is  example code from https://pytorch-forecasting.readthedocs.io/en/latest/tutorials/stallion.html

In [ ]:
from pytorch_forecasting.data.examples import get_stallion_data

data = get_stallion_data()

это перевести в 

In [ ]:
all_pumps_df

'2019-07-31 23:55:00'

In [41]:
training_oh["DT"] = pd.to_datetime(training_oh["DT"])

todo продолжить с этого места
тут получилось перевести в интеджер

In [52]:
test_column = training_oh["DT"] - min(training_oh["DT"])
test_column.astype('timedelta64[5m]').astype('int')

0         9999
1         9998
2         9997
3         9996
4         9995
          ... 
160257       4
160258       3
160259       2
160260       1
160261       0
Name: DT, Length: 160262, dtype: int64

In [43]:
training_oh["time_idx"] = training_oh["DT"].dt.day * 1440 + training_oh["DT"].dt.month * 1440 *12

KeyError: ignored

In [ ]:
data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month
data["time_idx"] -= data["time_idx"].min()

In [ ]:
data.time_idx

0        0
238      0
237      0
236      0
235      0
        ..
6765    59
6764    59
6763    59
6771    59
6650    59
Name: time_idx, Length: 21000, dtype: int64

In [ ]:

training = TimeSeriesDataSet(
    training_oh[lambda x: x.DT < training_cutoff],
    time_idx= 'DT',
    target= 'DSHORTT1138P2300058',
    group_ids=['WELL_ID'],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=[ 226000188,             226000198,
                   226000297,             226001846,             226001947,
                   226002142,             226002148,             226002307,
                   226002416,             226002603,             226002627,
                   226002861,             226002998,             226003074,
                   226003080,             226003316,             226003330 ],
    time_varying_known_reals=[ 'T1138P6000096',
             'T1138P6000315',   'DMIDT1138P4000064', 'DSHORTT1138P4000064',
        'DLONGT1138P4000064',   'DMIDT1138P2600012', 'DSHORTT1138P2600012',
        'DLONGT1138P2600012',   'DMIDT1205P2300000', 'DSHORTT1205P2300000',
        'DLONGT1205P2300000',       'T1205P2300000',       'T1138P4000064',
             'T1138P2600012',        'T1138P600050',        'T1013P500399' ]
)


AssertionError: ignored

In [ ]:

# create validation and training dataset
validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=2)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=2)


In [ ]:

# define trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
lr_logger = LearningRateMonitor()
trainer = pl.Trainer(
    max_epochs=100,
    gpus=0,
    gradient_clip_val=0.1,
    limit_train_batches=30,
    callbacks=[lr_logger, early_stop_callback],
)


In [ ]:

# create the model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=7,
    loss=QuantileLoss(),
    log_interval=2,
    reduce_on_plateau_patience=4
)


In [ ]:

print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


In [ ]:

# find optimal learning rate (set limit_train_batches to 1.0 and log_interval = -1)
res = trainer.tuner.lr_find(
    tft, train_dataloader=train_dataloader, val_dataloaders=val_dataloader, early_stop_threshold=1000.0, max_lr=0.3,
)

print(f"suggested learning rate: {res.suggestion()}")

In [ ]:

fig = res.plot(show=True, suggest=True)
fig.show()


In [ ]:

# fit the model
trainer.fit(
    tft, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
)